Imports

In [13]:
import sys
sys.path.append("../src")
import os
from functions import *
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from torch import save

In [14]:
# Get all HTML files from raw directory

filename_list = ["../data/raw_files/html/"+f for f in os.listdir('../data/raw_files/html')]

text_content_list = []
image_content_list = []
for filename in filename_list:

    with open(filename, 'r', encoding='utf-8') as file:
        html_content = file.read()

    text_content_list.extend(parse_html_content(html_content))
    image_content_list.extend(parse_html_images(html_content))

In [15]:
print(len(text_content_list))
print(len(image_content_list))

86
0


In [16]:
text_list = []
for content in text_content_list:
    # concatenate title and section header
    section = content['section'] + ": "
    # append text from paragraph to fill CLIP's 256 sequence limit
    text = section + content['text'][:256-len(section)]
    
    text_list.append(text)

image_list = []
for content in image_content_list:
    image_list.append(Image.open(content['image_path']))


In [17]:
print(len(text_list))
print(len(image_list))

86
0


In [18]:
# import model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")

# import processor (handles text tokenization and image preprocessing)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16") 

In [19]:
# pre-process text and images
inputs = processor(text=text_list, images=image_list, return_tensors="pt", padding=True)

IndexError: list index out of range

In [ ]:
# compute embeddings with CLIP
outputs = model(**inputs)

In [ ]:
# store embeddings in single torch tensor
text_embeddings = outputs.text_embeds
image_embeddings = outputs.image_embeds

In [ ]:
print(text_embeddings.shape)
print(image_embeddings.shape)

In [ ]:
# save content list as JSON
save_to_json(text_content_list, output_file='data/processed_files/JSON/text_content.json')
save_to_json(image_content_list, output_file='data/processed_files/JSON/image_content.json')

In [ ]:
# save embeddings to file
save(text_embeddings, 'data/processed_files/embeddings/text_embeddings.pt')
save(image_embeddings, 'data/processed_files/embeddings/image_embeddings.pt')